In [ ]:
import numpy as np
import pandas as pd
import pathlib as pl

import geopandas as gpd
import flopy
import itertools

import pyswmm
from pyswmm.swmm5 import PySWMM
from pyswmm import Simulation, Nodes

from modflowapi import ModflowApi
from modflowapi.extensions import ApiSimulation

In [ ]:
swmm_ws = pl.Path('../swmm/greenport')
mf_ws = pl.Path('../modflow/greenport_chd/base/')

In [ ]:
# MODFLOW -----------------
# load modflow model to create modelgrid polygons
mf_sim = flopy.mf6.MFSimulation.load(sim_ws=mf_ws, load_only=[], verbosity_level=0)
gwf = gwf = mf_sim.get_model(mf_sim.model_names[0])
mg = gwf.modelgrid
# model crs is EPSG:4456
grid_poly = mg.geo_dataframe.set_crs('epsg:4456')
grid_poly['cid'] = [(r, c) for r,c in itertools.product(range(mg.nrow), range(mg.ncol))]
grid_poly['bot01'] = mg.botm[1].ravel()

# manhole shapefile used in SWMM model
manhole_path = swmm_ws / 'Manhole_elevations.zip!Manhole_elevations/Manhole_elevation.shp'
manholes = gpd.read_file(manhole_path.as_posix()).to_crs(grid_poly.crs)

# SWMM junctions you want to couple with modflow
n_junctions = 50
junctions = manholes.sample(n_junctions,random_state=0)['Name'].values

# slice the manholes to grab desired junctions, join with model grid
mf6_swmm_connect = manholes.loc[manholes['Name'].isin(junctions), ['Name','geometry']] \
                    .sjoin(grid_poly)[['Name','cid','bot01']].set_index('Name')
mf6_cells = mf6_swmm_connect['cid'].to_dict()

In [ ]:
# Initialize SWMM

swmm_path = swmm_ws / "Case2.inp"
for suff in ['.out', '.rpt']:
    fpth = swmm_path.with_suffix(suff)
    if fpth.is_file():
        print(f"removing: '{fpth}'")
        fpth.unlink()
        
with Simulation(str(swmm_path)) as swmm_sim:
    mf6_swmm_connect['swmm_node'] = [pyswmm.Nodes(swmm_sim)[j] for j in mf6_swmm_connect.index]
    mf6_swmm_connect['invert_elev_ft'] = [node.invert_elevation for node in mf6_swmm_connect['swmm_node']]
    # swmm_nodes = {j: pyswmm.Nodes(swmm_sim)[j] for j in junctions}
    # swmm_invert_elevations = {key: val.invert_elevation for key, val in swmm_nodes.items()}
    assert np.all(mf6_swmm_connect['bot01'] < mf6_swmm_connect['invert_elev_ft'])
    
    dt = np.full((90), 86400, dtype=int)
    
    swmm_sim.step_advance(86400)
    
    swmm_sim.start()
    
    for t in dt:
        t0 = swmm_sim.current_time
        swmm_sim.__next__()
        t1 = swmm_sim.current_time
        print(f"start: {t0} end: {t1}")
        
    swmm_sim.terminate_simulation()
    swmm_sim.report()